In [2]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd

In [2]:
yf.pdr_override()
pdr.get_data_yahoo("AMZN", start="2020-03-10", end="2024-04-18").to_csv("amazon_stock.csv")

[*********************100%%**********************]  1 of 1 completed


In [3]:
layoffs = pd.read_csv(r"layoffs/layoffs.csv")
layoffs

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,The Good Glamm Group,New Delhi,Retail,150.0,0.15,2024-04-18,Unknown,India,180.0
1,Stability AI,London,AI,20.0,NaN,2024-04-18,Unknown,United Kingdom,174.0
2,ConnectWise,Tampa Bay,Other,NaN,NaN,2024-04-17,Acquired,United States,1.0
3,Google,SF Bay Area,Consumer,NaN,NaN,2024-04-17,Post-IPO,United States,26.0
4,Rivian,Detroit,Transportation,NaN,0.01,2024-04-17,Post-IPO,United States,10700.0
...,...,...,...,...,...,...,...,...,...
3563,Service,Los Angeles,Travel,NaN,1.00,2020-03-16,Seed,United States,5.1
3564,HopSkipDrive,Los Angeles,Transportation,8.0,0.10,2020-03-13,Unknown,United States,45.0
3565,Panda Squad,SF Bay Area,Consumer,6.0,0.75,2020-03-13,Seed,United States,1.0
3566,Tamara Mellon,Los Angeles,Retail,20.0,0.40,2020-03-12,Series C,United States,90.0


In [4]:
symbols = pd.read_csv(r"nasdaq_screener.csv")
symbols2 = pd.read_csv("S&P 500 Companies (Standard and Poor 500) - basics.csv")
symbols

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$139.23,1.7100,1.243%,4.080209e+10,United States,1999.0,701427,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$36.575,0.6350,1.767%,6.567370e+09,United States,2016.0,2937634,Industrials,Aluminum
2,AACG,ATA Creativity Global American Depositary Shares,$0.92,0.0192,2.131%,2.944012e+07,China,2008.0,11093,Real Estate,Other Consumer Services
3,AACI,Armada Acquisition Corp. I Common Stock,$11.74,0.1200,1.033%,0.000000e+00,United States,2021.0,9738,Finance,Blank Checks
4,AACIW,Armada Acquisition Corp. I Warrant,$0.14,0.0000,0.00%,0.000000e+00,United States,2021.0,10271,Finance,Blank Checks
...,...,...,...,...,...,...,...,...,...,...,...
7130,ZVRA,Zevra Therapeutics Inc. Common Stock,$4.79,0.0400,0.842%,2.080114e+08,United States,NaN,132607,Health Care,Biotechnology: Pharmaceutical Preparations
7131,ZVSA,ZyVersa Therapeutics Inc. Common Stock,$5.54,-0.2000,-3.484%,4.207552e+06,United States,2022.0,368385,Health Care,Biotechnology: Pharmaceutical Preparations
7132,ZWS,Zurn Elkay Water Solutions Corporation Common ...,$31.74,0.1100,0.348%,5.481398e+09,United States,2012.0,303610,Industrials,Industrial Machinery/Components
7133,ZYME,Zymeworks Inc. Common Stock,$9.165,0.1650,1.833%,6.467578e+08,United States,NaN,361393,Health Care,Biotechnology: Pharmaceutical Preparations


In [7]:
# attempt to map stock symbols on dataframe
layoffs["join"] = 1
symbols["join"] = 1
df_full = layoffs.merge(symbols, on="join").drop("join", axis=1)

symbols.drop("join", axis=1, inplace=True)

df_full['match'] = df_full.apply(
    lambda x: x["Name"].find(x['company']), axis=1).ge(0)

print(df_full[df_full['match']])

           company       location        industry  total_laid_off  \
34023       Rivian        Detroit  Transportation             NaN   
42154     Take-Two  New York City        Consumer           579.0   
56404        Tesla         Austin  Transportation         14000.0   
58747       Criteo          Paris       Marketing           140.0   
81250       Fabric  New York City       Logistics            30.0   
...            ...            ...             ...             ...   
25428489   Service    Los Angeles          Travel             NaN   
25428562   Service    Los Angeles          Travel             NaN   
25428596   Service    Los Angeles          Travel             NaN   
25428973   Service    Los Angeles          Travel             NaN   
25429105   Service    Los Angeles          Travel             NaN   

          percentage_laid_off        date     stage        country  \
34023                    0.01  2024-04-17  Post-IPO  United States   
42154                    0.05  

In [8]:
df_full[df_full['match']].to_csv("layoffs_and_stock_info.csv")

In [11]:
layoffs_and_stock_df = pd.read_csv("layoffs_and_stock_info.csv")

In [20]:
# clean up
layoffs_and_stock_df.drop(columns="Unnamed: 0").loc[layoffs_and_stock_df["Country"]==layoffs_and_stock_df["country"]].drop_duplicates(subset="Name").to_csv('layoffs_and_stock_info_CLEANED.csv')

Pull stock data for companies

In [23]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import os

In [24]:
start_date = "2020-03-10"
end_date = "2024-04-18"
layoffs_and_stock_df_clean = pd.read_csv("layoffs_and_stock_info_CLEANED.csv")

In [25]:
yf.pdr_override()
for i, row in layoffs_and_stock_df_clean.iterrows():
    symbol = row["Symbol"]
    industry = row["industry"]

    directory = "stocks\\" + industry

    if not os.path.exists(directory):
        os.makedirs(directory)

    pdr.get_data_yahoo(symbol, start=start_date, end=end_date).to_csv(directory + f"\\{symbol}.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Make layoffs datasets

In [1]:
import pandas as pd
import os

In [2]:
start_date = "2020-03-10"
end_date = "2024-04-18"
layoffs_and_stock_df_clean = pd.read_csv("layoffs_and_stock_info_CLEANED.csv")

In [3]:
layoffs_and_stock_df_clean

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng,Symbol,Date_datetime
0,3262,Inspirato,Denver,USA,North America,130.0,2020-03-16,22.0,591.0,461.0,Travel,Series C,$79,2020,39.73915,-104.98470,ISPO,2020-03-16
1,3239,Leafly,Seattle,USA,North America,91.0,2020-03-23,50.0,182.0,91.0,Retail,Acquired,$2,2020,47.60621,-122.33207,LFLY,2020-03-23
2,3237,Compass,New York City,USA,North America,375.0,2020-03-23,15.0,2500.0,2125.0,Real Estate,Series G,$1600,2020,40.71427,-74.00597,COMP,2020-03-23
3,3229,Sonder,San Francisco Bay Area,USA,North America,400.0,2020-03-24,33.0,1212.0,812.0,Travel,Series D,$359,2020,37.77493,-122.41942,SOND,2020-03-24
4,3193,Mogo,Vancouver,Canada,North America,78.0,2020-03-27,30.0,260.0,182.0,Finance,Post-IPO,$201,2020,49.24966,-123.11934,MOGO,2020-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,3309,Bumble,Austin,USA,North America,350.0,2024-02-27,30.0,1167.0,817.0,Consumer,Post-IPO,$313,2024,30.26715,-97.74306,BMBL,2024-02-27
292,3304,Electronic Arts,San Francisco Bay Area,USA,North America,670.0,2024-02-28,5.0,13400.0,12730.0,Consumer,Post-IPO,$2,2024,37.77493,-122.41942,EA,2024-02-28
293,3305,Vacasa,Portland,USA,North America,320.0,2024-02-28,5.0,6400.0,6080.0,Travel,Post-IPO,$834,2024,45.52345,-122.67621,VCSA,2024-02-28
294,3295,Edgio,Phoenix,USA,North America,80.0,2024-03-04,10.0,800.0,720.0,Infrastruc…,Post-IPO,$462,2024,33.44838,-112.07404,EGIO,2024-03-04


In [39]:
layoffs_and_stock_df_clean[["date","total_laid_off","percentage_laid_off"]].groupby("date").agg({"total_laid_off":"sum","percentage_laid_off":"mean"})

,total_laid_off,percentage_laid_off
date,,
1/10/2023,950.0,0.205
1/10/2024,330.0,0.080
1/11/2024,340.0,0.100
1/12/2023,160.0,0.105
1/12/2024,0.0,NaN
...,...,...
9/21/2023,0.0,NaN
9/22/2023,30.0,NaN
9/5/2023,30.0,0.150


In [37]:
for industry in layoffs_and_stock_df_clean['industry']:

    directory = "layoffs\\" + industry

    if not os.path.exists(directory):
        os.makedirs(directory)

    df = layoffs_and_stock_df_clean.loc[layoffs_and_stock_df_clean['industry'] == industry]

    df[["date","total_laid_off","percentage_laid_off"]].groupby("date").agg({"total_laid_off":"sum","percentage_laid_off":"mean"}).to_csv(directory + f"\\layoffs.csv")

Make Combined

In [4]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import os

In [5]:
start_date = "2020-03-10"
end_date = "2024-04-18"
# layoffs_and_stock_df_clean = pd.read_csv("layoffs_and_stock_info_CLEANED.csv", parse_dates=['date'])
layoffs_and_stock_df_clean = pd.read_csv("tech_layoffs_CLEANED_NoNA_Company_Size.csv", parse_dates=['Date_layoffs'])

In [6]:
layoffs_and_stock_df_clean

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng,Symbol,Date_datetime
0,3262,Inspirato,Denver,USA,North America,130.0,2020-03-16,22.0,591.0,461.0,Travel,Series C,$79,2020,39.73915,-104.98470,ISPO,2020-03-16
1,3239,Leafly,Seattle,USA,North America,91.0,2020-03-23,50.0,182.0,91.0,Retail,Acquired,$2,2020,47.60621,-122.33207,LFLY,2020-03-23
2,3237,Compass,New York City,USA,North America,375.0,2020-03-23,15.0,2500.0,2125.0,Real Estate,Series G,$1600,2020,40.71427,-74.00597,COMP,2020-03-23
3,3229,Sonder,San Francisco Bay Area,USA,North America,400.0,2020-03-24,33.0,1212.0,812.0,Travel,Series D,$359,2020,37.77493,-122.41942,SOND,2020-03-24
4,3193,Mogo,Vancouver,Canada,North America,78.0,2020-03-27,30.0,260.0,182.0,Finance,Post-IPO,$201,2020,49.24966,-123.11934,MOGO,2020-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,3309,Bumble,Austin,USA,North America,350.0,2024-02-27,30.0,1167.0,817.0,Consumer,Post-IPO,$313,2024,30.26715,-97.74306,BMBL,2024-02-27
292,3304,Electronic Arts,San Francisco Bay Area,USA,North America,670.0,2024-02-28,5.0,13400.0,12730.0,Consumer,Post-IPO,$2,2024,37.77493,-122.41942,EA,2024-02-28
293,3305,Vacasa,Portland,USA,North America,320.0,2024-02-28,5.0,6400.0,6080.0,Travel,Post-IPO,$834,2024,45.52345,-122.67621,VCSA,2024-02-28
294,3295,Edgio,Phoenix,USA,North America,80.0,2024-03-04,10.0,800.0,720.0,Infrastruc…,Post-IPO,$462,2024,33.44838,-112.07404,EGIO,2024-03-04


In [15]:
yf.pdr_override()
for industry in layoffs_and_stock_df_clean['industry']:
    directory = "combined_same_length\\" + industry

    if not os.path.exists(directory):
        os.makedirs(directory)

    df = layoffs_and_stock_df_clean.loc[layoffs_and_stock_df_clean['industry'] == industry]

    layoffs = df[["date","total_laid_off","percentage_laid_off"]].groupby("date").agg({"total_laid_off":"sum","percentage_laid_off":"mean"})
    for i, row in df.iterrows():
        symbol = row["Symbol"]

        stocks_df = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)
        if len(stocks_df) == 1034:
            merge = pd.merge(layoffs, stocks_df, left_index=True, right_index=True ,how="outer")
            merge['total_laid_off'] = merge['total_laid_off'].fillna(0)
            merge["percentage_laid_off"] = merge['percentage_laid_off'].fillna(0)
            merge.to_csv(directory + f"\\{symbol}.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['LFLYW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 comple

In [11]:
yf.pdr_override()
for industry in layoffs_and_stock_df_clean['industry']:
    directory = "separated\\" + industry

    if not os.path.exists(directory):
        os.makedirs(directory)

    df = layoffs_and_stock_df_clean.loc[layoffs_and_stock_df_clean['industry'] == industry].set_index("date")
    for symbol in df['Symbol'].unique():

        stocks_df = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)
        merged = pd.merge(df.loc[df['Symbol']==symbol][['total_laid_off', 'percentage_laid_off']], stocks_df, left_index=True, right_index=True,how="outer")
        merged['total_laid_off'] = merged['total_laid_off'].fillna(0)
        merged['percentage_laid_off'] = merged['percentage_laid_off'].fillna(0)
        merged.to_csv(directory + f"\\{symbol}.csv")

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['LFLYW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 compl

Make with company size

In [2]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import os
import numpy as np

In [4]:
start_date = "2020-03-10"
end_date = "2024-03-30"
layoffs_and_stock_df_clean = pd.read_csv("tech_layoffs_CLEANED_NoNA_Company_Size.csv", parse_dates=['Date_layoffs'])

In [5]:
layoffs_and_stock_df_clean

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng,Symbol,Date_datetime
0,3262,Inspirato,Denver,USA,North America,130.0,2020-03-16,22.0,591.0,461.0,Travel,Series C,$79,2020,39.73915,-104.98470,ISPO,2020-03-16
1,3239,Leafly,Seattle,USA,North America,91.0,2020-03-23,50.0,182.0,91.0,Retail,Acquired,$2,2020,47.60621,-122.33207,LFLY,2020-03-23
2,3237,Compass,New York City,USA,North America,375.0,2020-03-23,15.0,2500.0,2125.0,Real Estate,Series G,$1600,2020,40.71427,-74.00597,COMP,2020-03-23
3,3229,Sonder,San Francisco Bay Area,USA,North America,400.0,2020-03-24,33.0,1212.0,812.0,Travel,Series D,$359,2020,37.77493,-122.41942,SOND,2020-03-24
4,3193,Mogo,Vancouver,Canada,North America,78.0,2020-03-27,30.0,260.0,182.0,Finance,Post-IPO,$201,2020,49.24966,-123.11934,MOGO,2020-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,3309,Bumble,Austin,USA,North America,350.0,2024-02-27,30.0,1167.0,817.0,Consumer,Post-IPO,$313,2024,30.26715,-97.74306,BMBL,2024-02-27
292,3304,Electronic Arts,San Francisco Bay Area,USA,North America,670.0,2024-02-28,5.0,13400.0,12730.0,Consumer,Post-IPO,$2,2024,37.77493,-122.41942,EA,2024-02-28
293,3305,Vacasa,Portland,USA,North America,320.0,2024-02-28,5.0,6400.0,6080.0,Travel,Post-IPO,$834,2024,45.52345,-122.67621,VCSA,2024-02-28
294,3295,Edgio,Phoenix,USA,North America,80.0,2024-03-04,10.0,800.0,720.0,Infrastructure,Post-IPO,$462,2024,33.44838,-112.07404,EGIO,2024-03-04


In [6]:
yf.pdr_override()
for industry in layoffs_and_stock_df_clean['Industry']:
    directory = "separated_company_size_same_length\\" + industry

    if not os.path.exists(directory):
        os.makedirs(directory)

    df = layoffs_and_stock_df_clean.loc[layoffs_and_stock_df_clean['Industry'] == industry].set_index("Date_layoffs")
    for symbol in df['Symbol'].unique():

        stocks_df = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)
        if len(stocks_df) == 1021:
            merged = pd.merge(df.loc[df['Symbol']==symbol][['Company_Size_before_Layoffs', 'Company_Size_after_layoffs']], stocks_df, left_index=True, right_index=True,how="outer")
            merged['company_size'] = (merged['Company_Size_before_Layoffs'].shift(-1).fillna(0) + merged['Company_Size_after_layoffs'].fillna(0)).replace(0, np.nan)
            merged.loc[merged.index.min(), 'company_size'] = merged['company_size'][merged['company_size'].first_valid_index()]
            merged['company_size'] = merged['company_size'].interpolate()
            merged.to_csv(directory + f"\\{symbol}.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******